# 7 数据清洗与准备
重点讨论缺失值、重复值、字符串操作和其他数据转换工具

## 7.1 处理缺失值

In [1]:
# pandas对象的所有描述性统计信息默认情况下是排除缺失值的
import pandas as pd
import numpy as np

string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])
string_data

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [2]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

In [3]:
# NA意思是not available（不可用）
# 当数据清洗时，对缺失数据本身进行分析已确定数据收集问题或数据丢失导致的数据偏差通常很重要。
# None在对象数组中也被当作NA处理
string_data[0] = None
string_data

0         None
1    artichoke
2          NaN
3      avocado
dtype: object

In [4]:
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

### 7.1.1 过滤缺失值

In [5]:
from numpy import nan as NA

data = pd.Series([NA, 1, 2, NA, 4])
# Series中使用dropna，会返回所有非NA的数据
data.dropna()

1    1.0
2    2.0
4    4.0
dtype: float64

In [6]:
data[data.notnull()]

1    1.0
2    2.0
4    4.0
dtype: float64

In [7]:
# dropna默认删除包含NA的行
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA], [NA, NA, NA], [NA, 6.55, 3.]])
cleaned = data.dropna()
data

,0,1,2
0,1.0,6.50,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.55,3.0


In [8]:
cleaned

,0,1,2
0,1.0,6.5,3.0


In [9]:
# 当传入how='all'时，会删除所有值均为NA的行
data.dropna(how='all')

,0,1,2
0,1.0,6.50,3.0
1,1.0,NaN,NaN
3,NaN,6.55,3.0


In [10]:
data[4] = NA

In [11]:
data

,0,1,2,4
0,1.0,6.50,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.55,3.0,NaN


In [12]:
# 删除列加上axis=1
data.dropna(axis=1, how='all')

,0,1,2
0,1.0,6.50,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.55,3.0


In [13]:
# 只想保留一定数量的行，使用thresh参数来表示
df = pd.DataFrame(np.random.randn(7, 3))
df.iloc[:4, 1] = NA
df.iloc[:2, 2] = NA
df

,0,1,2
0,-0.954475,NaN,NaN
1,0.560101,NaN,NaN
2,-1.488223,NaN,0.748316
3,0.409183,NaN,0.248550
4,0.738554,-0.092121,0.102685
5,1.743622,0.574994,-1.019001
6,1.808631,2.624611,1.422054


In [14]:
df.dropna()

,0,1,2
4,0.738554,-0.092121,0.102685
5,1.743622,0.574994,-1.019001
6,1.808631,2.624611,1.422054


In [15]:
df.dropna(thresh=2)

,0,1,2
2,-1.488223,NaN,0.748316
3,0.409183,NaN,0.248550
4,0.738554,-0.092121,0.102685
5,1.743622,0.574994,-1.019001
6,1.808631,2.624611,1.422054


### 7.1.2 补全缺失值

In [16]:
# 大多数情况下使用fillna来不全缺失值
df.fillna(0)

,0,1,2
0,-0.954475,0.000000,0.000000
1,0.560101,0.000000,0.000000
2,-1.488223,0.000000,0.748316
3,0.409183,0.000000,0.248550
4,0.738554,-0.092121,0.102685
5,1.743622,0.574994,-1.019001
6,1.808631,2.624611,1.422054


In [17]:
# 可以使用字典来为不同的列填充不同的值
df.fillna({1: 0.5, 2: 1})

,0,1,2
0,-0.954475,0.500000,1.000000
1,0.560101,0.500000,1.000000
2,-1.488223,0.500000,0.748316
3,0.409183,0.500000,0.248550
4,0.738554,-0.092121,0.102685
5,1.743622,0.574994,-1.019001
6,1.808631,2.624611,1.422054


In [18]:
# fillna默认不会修改原有的对象，加inplace=True会就地修改
_ = df.fillna({1: 0.5, 2: 1}, inplace=True)
df

,0,1,2
0,-0.954475,0.500000,1.000000
1,0.560101,0.500000,1.000000
2,-1.488223,0.500000,0.748316
3,0.409183,0.500000,0.248550
4,0.738554,-0.092121,0.102685
5,1.743622,0.574994,-1.019001
6,1.808631,2.624611,1.422054


In [19]:
df = pd.DataFrame(np.random.randn(6, 3))
df.iloc[2:, 1] = NA
df.iloc[4:, 2] = NA
df

,0,1,2
0,0.938774,1.499679,1.445087
1,0.140681,-1.651379,-2.044824
2,0.578811,NaN,-0.484238
3,1.200117,NaN,0.453228
4,-0.419241,NaN,NaN
5,0.061593,NaN,NaN


In [20]:
# 使用前一项值来填充空值
df.fillna(method='ffill')

,0,1,2
0,0.938774,1.499679,1.445087
1,0.140681,-1.651379,-2.044824
2,0.578811,-1.651379,-0.484238
3,1.200117,-1.651379,0.453228
4,-0.419241,-1.651379,0.453228
5,0.061593,-1.651379,0.453228


In [21]:
# 也可以使用均值等来填充缺失值
df.fillna(df.mean())

,0,1,2
0,0.938774,1.499679,1.445087
1,0.140681,-1.651379,-2.044824
2,0.578811,-0.075850,-0.484238
3,1.200117,-0.075850,0.453228
4,-0.419241,-0.075850,-0.157687
5,0.061593,-0.075850,-0.157687


## 7.2 数据转换

### 7.2.1 删除重复值

In [22]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'], 'k2': [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [23]:
# 返回行是否重复
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [24]:
# 删除重复的行
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


In [25]:
# duplicated和drop_duplicates默认都是保留地一个观测到的值
# 传入keep='last'就会保留最后一个
data.drop_duplicates(keep='last')

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
6,two,4


### 7.2.2 使用函数或Map进行数据转换

In [26]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                              'Pastrami', 'corned beef', 'Bacon',
                              'pastrami', 'honey ham', 'nova lox'],
                     'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [27]:
# 想要添加一列用于表明每种食物的动物肉类型，先写一个Map
meat_to_animal = {'bacon': 'pig',
                  'pulled pork': 'pig',
                  'pastrami': 'cow',
                  'corned beef': 'cow',
                  'honey ham': 'pig',
                  'nova lox': 'salmon'}

In [28]:
lowercased = data['food'].str.lower()
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [29]:
data['animal'] = lowercased.map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [30]:
# 使用匿名函数，来完成转换工作
data['animal1'] = data['food'].map(lambda x: meat_to_animal[x.lower()])
data

,food,ounces,animal,animal1
0,bacon,4.0,pig,pig
1,pulled pork,3.0,pig,pig
2,bacon,12.0,pig,pig
3,Pastrami,6.0,cow,cow
4,corned beef,7.5,cow,cow
5,Bacon,8.0,pig,pig
6,pastrami,3.0,cow,cow
7,honey ham,5.0,pig,pig
8,nova lox,6.0,salmon,salmon


### 7.2.3 替代值

In [31]:
# 使用replace
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [32]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [33]:
# 一次性替代多个值
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [34]:
# 将不同的值替换为不同的值
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

### 7.2.3 重命名轴索引

In [35]:
data = pd.DataFrame(np.arange(12).reshape(3, 4),
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [36]:
# 转换df的轴索引
transform = lambda x: x[:4].upper()
data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [37]:
# 如果想要创建数据集转换后的版本，并且不修改原有的数据集，一个有用的方法时rename
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


In [38]:
# 想要修改原有数据集
data.rename(index={'OHIO': 'INDIANA'}, inplace=True)
data

,one,two,three,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


### 7.2.5 离散化和分箱

In [39]:
ages = [20, 22, 25, 27, 21, 23, 27, 37, 31, 61, 45, 41, 32]
bins = [18, 25, 35, 60, 100]
cate = pd.cut(ages, bins)
cate

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 13
Categories (4, interval[int64, right]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [40]:
# 返回分箱名的字符串数组
cate.codes

array([0, 0, 0, 1, 0, 0, 1, 2, 1, 3, 2, 2, 1], dtype=int8)

In [41]:
cate.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]], dtype='interval[int64, right]')

In [42]:
# 统计各区间的数量
pd.value_counts(cate)

(18, 25]     5
(25, 35]     4
(35, 60]     3
(60, 100]    1
dtype: int64

In [43]:
# 使用right=False来改变哪一边是封闭的
pd.cut(ages, [18, 25, 35, 60, 100], right=False)

[[18, 25), [18, 25), [25, 35), [25, 35), [18, 25), ..., [25, 35), [60, 100), [35, 60), [35, 60), [25, 35)]
Length: 13
Categories (4, interval[int64, left]): [[18, 25) < [25, 35) < [35, 60) < [60, 100)]

In [44]:
# 可以自定义箱名
group_names = ['Youth', 'YouthAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels=group_names)

['Youth', 'Youth', 'Youth', 'YouthAdult', 'Youth', ..., 'YouthAdult', 'Senior', 'MiddleAged', 'MiddleAged', 'YouthAdult']
Length: 13
Categories (4, object): ['Youth' < 'YouthAdult' < 'MiddleAged' < 'Senior']

In [45]:
# 如果传递的bins是整数，pandas将根据数据中的最小值和最大值计算等长的箱
data = np.random.randn(20)
pd.cut(data, 4, precision=2)

[(0.24, 1.2], (-0.72, 0.24], (-1.68, -0.72], (0.24, 1.2], (-0.72, 0.24], ..., (-0.72, 0.24], (0.24, 1.2], (0.24, 1.2], (-0.72, 0.24], (1.2, 2.15]]
Length: 20
Categories (4, interval[float64, right]): [(-1.68, -0.72] < (-0.72, 0.24] < (0.24, 1.2] < (1.2, 2.15]]

In [46]:
# qcut是根据分位数进行分箱，取决于数据的分布，
# 使用cut通常不会获得相同长度的数据量的数据点
# qcut使用样本的分位数，可以获得等长的分箱
data = np.random.randn(1000)
cats = pd.qcut(data, 4)  # 切成4份
cats

[(0.745, 3.031], (-2.8289999999999997, -0.714], (0.745, 3.031], (-0.714, 0.0465], (0.0465, 0.745], ..., (0.0465, 0.745], (-2.8289999999999997, -0.714], (0.0465, 0.745], (0.745, 3.031], (-0.714, 0.0465]]
Length: 1000
Categories (4, interval[float64, right]): [(-2.8289999999999997, -0.714] < (-0.714, 0.0465] < (0.0465, 0.745] < (0.745, 3.031]]

In [47]:
pd.value_counts(cats)

(-2.8289999999999997, -0.714]    250
(-0.714, 0.0465]                 250
(0.0465, 0.745]                  250
(0.745, 3.031]                   250
dtype: int64

In [48]:
# 与cut类似，可以传入自定义个分位数
cats1 = pd.qcut(data, [0, 0.1, 0.5, 0.9, 1])

In [49]:
pd.value_counts(cats1)

(-1.249, 0.0465]                 400
(0.0465, 1.328]                  400
(-2.8289999999999997, -1.249]    100
(1.328, 3.031]                   100
dtype: int64

### 7.2.6 检测和过滤异常值

In [50]:
data = pd.DataFrame(np.random.randn(1000, 4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.035500,0.013730,0.026304,-0.017396
std,1.003912,1.019372,0.997710,1.018395
min,-3.169595,-2.831331,-3.363307,-3.300830
25%,-0.611417,-0.675094,-0.676740,-0.735576
50%,0.015055,-0.001313,0.030704,-0.040622
75%,0.646788,0.710349,0.705058,0.708862
max,3.483084,3.858230,2.786009,3.132130


In [51]:
# 找出绝对值大于3的值
col = data[2]
col[np.abs(col) > 3]

28    -3.363307
669   -3.190961
965   -3.300249
Name: 2, dtype: float64

In [52]:
# 找出绝对值大于3的所有行
data[(np.abs(data) > 3).any(1)]

,0,1,2,3
28,0.783476,-0.522153,-3.363307,-1.514563
154,-0.596334,0.089899,0.589129,-3.300830
291,-3.169595,0.464334,-0.344410,0.882462
304,3.363714,0.974934,-1.401184,0.129550
472,-0.059456,3.260350,-0.629480,-0.795999
669,-1.645080,1.010314,-3.190961,0.480563
692,3.483084,-1.652372,-0.083970,-0.695350
732,1.283405,0.303513,0.192854,3.132130
777,1.208218,3.858230,-0.069776,-1.174998
800,0.812514,1.298221,-1.835976,3.029131


In [53]:
# 将绝对值大于3的正负数分别转换为3和-3
# np.sign是将正数转换为1，负数转换为-1
data[np.abs(data) > 3] = np.sign(data) * 3
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.034285,0.012591,0.027159,-0.017257
std,0.999033,1.015630,0.994982,1.016980
min,-3.000000,-2.831331,-3.000000,-3.000000
25%,-0.611417,-0.675094,-0.676740,-0.735576
50%,0.015055,-0.001313,0.030704,-0.040622
75%,0.646788,0.710349,0.705058,0.708862
max,3.000000,3.000000,2.786009,3.000000


### 7.2.7 置换和随机抽样

In [54]:
df = pd.DataFrame(np.arange(5 * 4).reshape(5, 4))
# np.random.permutation生成一个随机序列
sampler = np.random.permutation(5)
type(sampler)

numpy.ndarray

In [55]:
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [56]:
df.take(sampler)

,0,1,2,3
3,12,13,14,15
1,4,5,6,7
0,0,1,2,3
2,8,9,10,11
4,16,17,18,19


In [57]:
# 返回一个随机子集
df.sample(3)

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11


In [58]:
# 要生成一个带有替代值的样本（允许有重复选择）
choice = pd.Series([5, -7, -1, 6, 4])
choice.sample(n=10, replace=True)

0    5
4    4
1   -7
3    6
3    6
1   -7
0    5
4    4
0    5
0    5
dtype: int64

### 7.2.8 计算指标和虚拟变量

In [59]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'], 'data1': range(6)})
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [60]:
# 根据一列内容k个不同的值，可以衍生出k列的值为0或1的矩阵或DataFrame
# 对应位置上有值则为1，否则为0
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [61]:
dummies = pd.get_dummies(df['key'], prefix='key')
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


In [62]:
# 如果一行属于多个类别，则比较复杂，如电影的分类
movie_name = ['movie_id', 'title', 'genres']
movies = pd.read_table("./examples/movies.dat", sep="::", header=None, names=movie_name)
movies[:10]

/var/folders/ml/8mffbtvx25b6ftr05mlfbtqm0000gn/T/ipykernel_20358/2418823012.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies = pd.read_table("./examples/movies.dat", sep="::", header=None, names=movie_name)


,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [63]:
# 首先提取出电影的流派
all_genres = []
for x in movies.genres:
    all_genres.extend(x.split('|'))
genres = pd.unique(all_genres)
genres.tolist()

['Animation',
 "Children's",
 'Comedy',
 'Adventure',
 'Fantasy',
 'Romance',
 'Drama',
 'Action',
 'Crime',
 'Thriller',
 'Horror',
 'Sci-Fi',
 'Documentary',
 'War',
 'Musical',
 'Mystery',
 'Film-Noir',
 'Western']

In [64]:
# 首先构建一个全0的DataFrame
zero_matrix = np.zeros((len(movies), len(genres)))
# 生成一个DataFrame
dummies = pd.DataFrame(zero_matrix, columns=genres)
dummies.head(5)

,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [65]:
# 遍历movies，将dummies中符合条件的条目置为1
for i, gen in enumerate(movies.genres):
    indices = dummies.columns.get_indexer(gen.split('|'))  # 获取值对应列名的索引
    dummies.iloc[i, indices] = 1

In [66]:
dummies.head(10)

,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [67]:
movies_windic = movies.join(dummies.add_prefix('Genre_'))
movies_windic.iloc[0]

movie_id                                       1
title                           Toy Story (1995)
genres               Animation|Children's|Comedy
Genre_Animation                              1.0
Genre_Children's                             1.0
Genre_Comedy                                 1.0
Genre_Adventure                              0.0
Genre_Fantasy                                0.0
Genre_Romance                                0.0
Genre_Drama                                  0.0
Genre_Action                                 0.0
Genre_Crime                                  0.0
Genre_Thriller                               0.0
Genre_Horror                                 0.0
Genre_Sci-Fi                                 0.0
Genre_Documentary                            0.0
Genre_War                                    0.0
Genre_Musical                                0.0
Genre_Mystery                                0.0
Genre_Film-Noir                              0.0
Genre_Western       

In [68]:
# 将get_dummies与cut结合使用
data = np.random.randn(10)
df = pd.DataFrame(data)
bins = [-np.inf, 0, 0.2, 0.4, 0.6, 0.8, 1, np.inf]
dummies_1 = pd.get_dummies(pd.cut(data, bins))

In [69]:
df.join(dummies_1)

,0,"(-inf, 0.0]","(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]","(1.0, inf]"
0,-0.889393,1,0,0,0,0,0,0
1,0.446608,0,0,0,1,0,0,0
2,1.343538,0,0,0,0,0,0,1
3,0.335843,0,0,1,0,0,0,0
4,3.473673,0,0,0,0,0,0,1
5,-0.450138,1,0,0,0,0,0,0
6,-0.177174,1,0,0,0,0,0,0
7,0.014415,0,1,0,0,0,0,0
8,-0.468825,1,0,0,0,0,0,0
9,0.509336,0,0,0,1,0,0,0


## 7.3 字符串操作

In [70]:
# 使用split方法拆分字符串
'a, b, guido'.split(",")

['a', ' b', ' guido']

In [71]:
val = 'a, b, guido'
pieces = [x.strip() for x in val.split(",")]
pieces

['a', 'b', 'guido']

In [72]:
first, second, third = pieces
first

'a'

In [73]:
val.count(",")

2

In [74]:
# 正则表达式
import re

text = "foo    bar\t baz   \tqux"
re.split('\s+', text)

['foo', 'bar', 'baz', 'qux']

In [75]:
# 如果需要将相同的表达式应用到多个字符串上，推荐使用re.compile创建一个正则表达式对西那个，这样做有利于节约CPU周期
regex = re.compile(text)
regex.findall(text)

['foo    bar\t baz   \tqux']

In [76]:
# 识别电子邮件的正则表达式
text = """
Dave dave@gmail.com
Steve steve@gmail.com
Rob rob@outlook.com
Ryan ryan@yahoo.com
"""
pattern = r'[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,4}'
regex = re.compile(pattern, flags=re.IGNORECASE)
regex.findall(text)

['dave@gmail.com', 'steve@gmail.com', 'rob@outlook.com', 'ryan@yahoo.com']

In [77]:
m = regex.search(text)
m

<re.Match object; span=(6, 20), match='dave@gmail.com'>

In [78]:
text[m.start():m.end()]

'dave@gmail.com'

In [79]:
# 假设想要查找电子邮件地址，并将用户名、域名和域名后缀分开，可以使用括号将模式包起来
pattern_1 = r'([A-Za-z0-9._%+-]+)@([A-Za-z0-9._]+)\.([A-Za-z]{2,4})'
regex = re.compile(pattern_1, flags=re.IGNORECASE)
regex.findall(text)

[('dave', 'gmail', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'outlook', 'com'),
 ('ryan', 'yahoo', 'com')]

In [80]:
m = regex.match('test@gmail.com')
m.groups()

('test', 'gmail', 'com')

In [81]:
# sub可以使用特殊符号
# \1表示第一个匹配分组，\2表示第二个匹配分组
print(regex.sub(r'Username:\1, Domain:\2, Suffix:\3', text))


Dave Username:dave, Domain:gmail, Suffix:com
Steve Username:steve, Domain:gmail, Suffix:com
Rob Username:rob, Domain:outlook, Suffix:com
Ryan Username:ryan, Domain:yahoo, Suffix:com



In [82]:
# pandas中向量化字符串函数
data = {'Dave': 'dave@google.com',
        'Steve': 'steve@outlook.com',
        'Rob': 'rob@gmail.com',
        'Wes': np.nan}
data = pd.Series(data)
data  # 电子邮件中有NA值

Dave       dave@google.com
Steve    steve@outlook.com
Rob          rob@gmail.com
Wes                    NaN
dtype: object

In [83]:
data.isnull()

Dave     False
Steve    False
Rob      False
Wes       True
dtype: bool

In [84]:
# 在处理NA值时，map和正则表达式会失效，为了解决这个问题，Series有面向数组的方法用于跳过NA值的字符串操作。
# 这些方法通过Series的str属性进行调用
data.str.contains("gmail")

Dave     False
Steve    False
Rob       True
Wes        NaN
dtype: object

In [85]:
# 可以结合正则表达式
data.str.findall(pattern_1, flags=re.IGNORECASE)

Dave       [(dave, google, com)]
Steve    [(steve, outlook, com)]
Rob          [(rob, gmail, com)]
Wes                          NaN
dtype: object

In [86]:
matchs = data.str.match(pattern_1, flags=re.IGNORECASE)
matchs

Dave     True
Steve    True
Rob      True
Wes       NaN
dtype: object

In [87]:
type(matchs)

pandas.core.series.Series

In [88]:
# 使用字符串切片的类似语法进行向量化切片
data.str[:5]

Dave     dave@
Steve    steve
Rob      rob@g
Wes        NaN
dtype: object